In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import data from ENTSOE

Warning! The data of ENTSOE is actualised every day at 14:00 CEST. The server isn't available at this time +- 5 minutes!

https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=GRAPH&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=04.04.2024%2000:00|CET%7CDAYTIMERANGE&dateTime.endDateTime=04.04.2024%2000:00|CET%7CDAYTIMERANGE&area.values=CTY%7C10Y1001A1001A83F!CTY%7C10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)

In [40]:
# Get the current date
current_date = datetime.now()
# Format the date as dd.mm.yyyy
formatted_date_today = current_date.strftime("%d.%m.%Y")
formatted_date_today = "03.04.2024"


# URL of the webpage containing the table
url = f"https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime={formatted_date_today}+00:00|CET|DAYTIMERANGE&dateTime.endDateTime={formatted_date_today}+00:00|CET|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)"

# Send a GET request to fetch the webpage content
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables in the HTML
tables = soup.find_all('table')

# Iterate through each table to find the one containing "Biomass"
target_table = None
for table in tables:
    if "Biomass" in table.get_text():
        target_table = table
        break

if target_table:
    # Convert the HTML table to a pandas DataFrame
    df = pd.read_html(str(target_table))[0]

    # Replace "n/e" with NaN
    df.replace('n/e', pd.NA, inplace=True)

    # Print the DataFrame
    # print(df)
    df.to_csv("../../data/carbon_footprint_data.csv", sep=';', header=True,index=False)
    
else:
    print("No table containing 'Biomass' found on the webpage.")

In [41]:
df = pd.read_csv("../../data/carbon_footprint_data.csv", sep=";",header=1)
df = df.iloc[3:]
# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)
df[['start', 'end']] = df['MTU'].str.split(' - ', expand=True)


# Get the current date and time in CET timezone
current_datetime_cet = datetime.now(pytz.timezone('Europe/Paris'))

# Extract the date part
current_date = current_datetime_cet.date()

# Convert the "start" column to datetime format with the current date
df['start_datetime'] = pd.to_datetime(current_date.strftime("%Y-%m-%d") + ' ' + df['start'])
df['start_datetime'] = df['start_datetime'].dt.tz_localize('Europe/Paris')

# Convert the datetime to Unix timestamp
df['start_unix_timestamp'] = df['start_datetime'].astype(int) / 10**9  # Convert nanoseconds to seconds

# Convert the "start" column to datetime format with the current date
df['end_datetime'] = pd.to_datetime(current_date.strftime("%Y-%m-%d") + ' ' + df['end'])
df['end_datetime'] = df['end_datetime'].dt.tz_localize('Europe/Paris')

# Convert the datetime to Unix timestamp
df['end_unix_timestamp'] = df['end_datetime'].astype(int) / 10**9  # Convert nanoseconds to seconds

df.to_csv("../../data/carbon_footprint_data.csv", sep=';', header=True,index=False)
